# Markov Decision Process

Die Grundlage für Reinforcement learning bilden Markow-Entscheidungsprozesse. Aus diesem Grund sollen jene nun genauer betrachtet werden.
Ein Markov Decision Process (MDP) oder zu Deutsch auch Markow-Entscheidungsprozess ist ein Fünf-Tupel der Form $(S, A, T, R, s_0) $
wobei gilt:

* $S$ = eine Menge von Zuständen (states)
* $A$ = eine Menge von Aktionen (actions)
* $T(s, a, s’)$ = eine transition function mit $s \in S$ 
* $R$ = Belohnungsfunktion $R(s, a, s')$
* $s_0$ = Startzustand

Ein MDP ist immer von seinem Umfeld, dem Environment abhänging und kann optional einen Endzustand besitzen. Die Lösung von einem solchen Problem ist eine Funktion der Form $\pi \colon S\rightarrow A$. Diese wird auch Policy genannt. Die Policy gibt an, welche Aktion in welchem Zustand ausgeführt wird. Ziel ist es, eine (möglichst) optimale Policy zu finden. (vgl. Abbeel und Klein)

Der Prozess hat dabei die Markov-Eigenschaft. Das bedeutet, dass der Folgezustand nur vom aktuellen Zustand abhänging ist und nicht auf den vorangegangenen basiert. Es gilt also:

$P(S_{t+1}=s_t'|s_t,a_t,s_{t-1},a_{t-1})  =  P(S_{t+1}=s_t'|s_t,a_t) \forall s \in S$

## Environment, states, actions und rewards

## Transition function, Probability und Reward function
Die Transition function gibt an, mit welcher Wahrscheinlichkeit man von einem Zustand in den nächsten gelangt. Es handelt sich also um eine Funktion der Art  $P_{ss'}(s'| s, a)$. Dabei is $P$ die Wahrscheinlichkeit, dass Aktion $a$ vom Zustand $s$ zu $s'$ führt). Das Ergebnis der Funktion kann mit Hilfe einer Matrix dargestellt werden:


$P = \begin{bmatrix}
P_{11} & ... & P_{1n}\\
... &  &\\
P_{n1} & ... & P_{nn}
\end{bmatrix}$ wobei $1$ bis $n$ alle möglichen Zustände bezeichnen

Die Summe der Werte aus einer Reihe der Matrix muss für alle Reihen 1 ergeben.

Für jeden Zustandswechsel kann eine Belohnung $r$ ("Reward") vergeben werden. Der Höhe der Belohung ist durch die Reward function festgelegt. In vielen MDPs wird die Belohnung mit einem Faktor $\gamma$, dem "Discount" multipliziert. Dabei ist $\gamma \in [0,1]$ und wird mit der Anzahl der bisher erfolgten Schritte $t$ potenziert. Der Discount-Faktor wird oftmals genutzt, um zu erreichen, das früher erhaltene Belohnungen stärker gewichtet werden als spätere. Der aktuelle Wert des Zustands ergibt sich aus der Summe der erhaltenen Belohungen mal des Discounts. Ziel ist es, den Wert zu maximieren.
Es gilt:

\\[\texttt{Wert} = \sum_{k = 0}^t \gamma^k\cdot r_{k+1}\\]

